In [1]:
import numpy as np
import pandas as pd
from scipy import stats

In [2]:
df= pd.read_csv("ab_data.csv")
df.head()

,user_id,timestamp,group,landing_page,converted
0,851104,2017-01-21 22:11:48.556739,control,old_page,0
1,804228,2017-01-12 08:01:45.159739,control,old_page,0
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0
4,864975,2017-01-21 01:52:26.210827,control,old_page,1


In [3]:
df.groupby(["group","landing_page"]).count()

user_id  timestamp  converted
group     landing_page                               
control   new_page         1928       1928       1928
          old_page       145274     145274     145274
treatment new_page       145311     145311     145311
          old_page         1965       1965       1965

In [4]:
mask1 = df["group"] == "control" 
mask2 = df["landing_page"] == "old_page"

control = df[mask1 & mask2]["converted"]

In [5]:
mask1 = df["group"] == "treatment" 
mask2 = df["landing_page"] == "new_page"

treatment = df[mask1 & mask2]["converted"]

Null Hypothesis: The difference of average conversion rate between new_page and old_page is zero


Alternative Hypothesis: The difference of average conversion rate between new_page and old_page is NOT zero


In [6]:
tt = stats.ttest_ind(treatment, control, equal_var=False)

In [7]:
tt.pvalue

0.18965416073470676

#### **P-Value is NOT statistically significant. We FAIL to reject the Null Hypothesis.**

---

There are a few countries. We can conduct our hypothesis testing based on countries

In [8]:
countries = pd.read_csv("countries.csv")

In [9]:
countries.head()

,user_id,country
0,834778,UK
1,928468,US
2,822059,UK
3,711597,UK
4,710616,UK


In [10]:
df = df.merge(countries, how="left", on="user_id")
df.head()

,user_id,timestamp,group,landing_page,converted,country
0,851104,2017-01-21 22:11:48.556739,control,old_page,0,US
1,804228,2017-01-12 08:01:45.159739,control,old_page,0,US
2,661590,2017-01-11 16:55:06.154213,treatment,new_page,0,US
3,853541,2017-01-08 18:28:03.143765,treatment,new_page,0,US
4,864975,2017-01-21 01:52:26.210827,control,old_page,1,US


In [16]:
# dropping control and new_page
mask1 = df["group"]=="control"
mask2 = df["landing_page"] == "new_page"

indexnames = df[mask1 & mask2].index
df.drop(index=indexnames, inplace=True)

In [18]:
# dropping treatment and old_page

mask1 = df["group"]=="treatment"
mask2 = df["landing_page"] == "old_page"

indexnames = df[mask1 & mask2].index
df.drop(index=indexnames, inplace=True)

In [19]:
df.shape

(290585, 6)

In [21]:
df["country"].unique()

array(['US', 'CA', 'UK'], dtype=object)

In [22]:
df.groupby("country").count()

,user_id,timestamp,group,landing_page,converted
country,,,,,
CA,14499,14499,14499,14499,14499
UK,72466,72466,72466,72466,72466
US,203620,203620,203620,203620,203620


In [33]:
ca_control = df[df["country"]=="CA"].loc[df["group"] == "control","converted"]
ca_treatment = df[df["country"]=="CA"].loc[df["group"] == "treatment","converted"]


**Null Hypothesis:** The difference of average conversion rate between new_page and old_page is zero in Canada


**Alternative Hypothesis:** The difference of average conversion rate between new_page and old_page is NOT zero in Canada

In [26]:
tt = stats.f_oneway(ca_treatment, ca_control)

In [27]:
tt.pvalue

0.19469116780847598

In [29]:
tt = stats.ttest_ind(ca_treatment, ca_control, equal_var=False)
tt.pvalue

0.1947734557196999

**WE FAIL TO REJECT THE NULL HYPOTHESIS** 

---

In [30]:
us_control = df[df["country"]=="US"].loc[df["group"] == "control","converted"]
us_treatment = df[df["country"]=="US"].loc[df["group"] == "treatment","converted"]


**Null Hypothesis:** The difference of average conversion rate between new_page and old_page is zero in the US


**Alternative Hypothesis:** The difference of average conversion rate between new_page and old_page is NOT zero in the US

In [32]:
stats.f_oneway(us_treatment, us_control)


F_onewayResult(statistic=2.268065499079801, pvalue=0.1320662107447698)

**WE FAIL TO REJECT THE NULL HYPOTHESIS** 

---

In [35]:
uk_control = df[df["country"]=="UK"].loc[df["group"] == "control","converted"]
uk_treatment = df[df["country"]=="UK"].loc[df["group"] == "treatment","converted"]

**Null Hypothesis:** The difference of average conversion rate between new_page and old_page is zero in the US


**Alternative Hypothesis:** The difference of average conversion rate between new_page and old_page is NOT zero in the US

In [36]:
stats.f_oneway(uk_treatment, uk_control)


F_onewayResult(statistic=0.22551604671569786, pvalue=0.6348701618976766)

The new landing page does not significantly improve the conversion rates in any given countries